### SQL Agent: Natural Language to SQL Query Generator

This notebook will teach you step-by-step how to build a robust SQL agent that converts natural language queries into PostgreSQL queries.

In [1]:
# Install required packages
%pip install -q psycopg2-binary pandas sqlalchemy openai anthropic python-dotenv langchain langchain-openai langchain-anthropic
%pip install -q ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
2+2

4

In [3]:
# Core libraries
import os
import re
import json
from typing import List, Dict, Any, Optional
import warnings
warnings.filterwarnings('ignore')

# Database libraries
import psycopg2
from psycopg2.extras import RealDictCursor
import pandas as pd
from sqlalchemy import create_engine, text, inspect

# AI libraries
import openai
from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

# Utility libraries
from datetime import datetime
import time

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [4]:
class PostgreSQLConnection:
    """
    A robust PostgreSQL connection handler with error handling and connection management.
    """
    
    def __init__(self, host, port, database, user, password):
        self.host = host
        self.port = port
        self.database = database
        self.user = user
        self.password = password
        self.connection = None
        self.engine = None
        
    def connect(self):
        """Establish connection to PostgreSQL database"""
        try:
            # Create connection string
            connection_string = f"postgresql://{self.user}:{self.password}@{self.host}:{self.port}/{self.database}"
            
            # Create SQLAlchemy engine
            self.engine = create_engine(connection_string)
            
            # Test connection
            with self.engine.connect() as conn:
                result = conn.execute(text("SELECT version()"))
                version = result.fetchone()[0]
                print(f"✅ Connected to PostgreSQL!")
                print(f"📊 Database: {self.database}")
                print(f"🔧 Version: {version[:50]}...")
                
            return True
            
        except Exception as e:
            print(f"❌ Connection failed: {str(e)}")
            return False
    
    def execute_query(self, query, return_df=True):
        """Execute a SQL query and return results"""
        try:
            if return_df:
                df = pd.read_sql_query(query, self.engine)
                return df
            else:
                with self.engine.connect() as conn:
                    result = conn.execute(text(query))
                    return result.fetchall()
                    
        except Exception as e:
            print(f"❌ Query execution failed: {str(e)}")
            return None
    
    def get_table_info(self):
        """Get information about all tables in the database"""
        try:
            inspector = inspect(self.engine)
            tables_info = {}
            
            for table_name in inspector.get_table_names():
                columns = inspector.get_columns(table_name)
                tables_info[table_name] = {
                    'columns': [col['name'] for col in columns],
                    'column_details': columns
                }
                
            return tables_info
            
        except Exception as e:
            print(f"❌ Failed to get table info: {str(e)}")
            return None

# Initialize database connection
DB_CONFIG = {
    'host': '54.251.218.166',
    'port': 5432,
    'database': 'dummy',
    'user': 'rajesh',
    'password': 'rajesh123'
}

# Create database connection
db = PostgreSQLConnection(**DB_CONFIG)
connection_success = db.connect()







✅ Connected to PostgreSQL!
📊 Database: dummy
🔧 Version: PostgreSQL 15.13 (Debian 15.13-1.pgdg120+1) on x86...


In [5]:
db.execute_query("SELECT * FROM actor a")

,actor_id,first_name,last_name,last_update
0,1,Penelope,Guiness,2013-05-26 14:47:57.620
1,2,Nick,Wahlberg,2013-05-26 14:47:57.620
2,3,Ed,Chase,2013-05-26 14:47:57.620
3,4,Jennifer,Davis,2013-05-26 14:47:57.620
4,5,Johnny,Lollobrigida,2013-05-26 14:47:57.620
...,...,...,...,...
195,196,Bela,Walken,2013-05-26 14:47:57.620
196,197,Reese,West,2013-05-26 14:47:57.620
197,198,Mary,Keitel,2013-05-26 14:47:57.620
198,199,Julia,Fawcett,2013-05-26 14:47:57.620
